In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import category_encoders as ce
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE




def load_data(file_path):
    """Load data from a CSV file into a pandas DataFrame."""
    return pd.read_csv(file_path)

In [2]:
df = load_data('data/bank_data_train.csv')
TRESHOLD = 0.7
print(df.shape)

(355190, 116)


In [3]:
def explore_data(df):
    """Display basic information about the DataFrame and drop 'ID' if present. Returns the (possibly) modified DataFrame."""
    print("DataFrame Info:")
    df.info()
    
    # drop unnecessary column if it exists
    if 'ID' in df.columns:
        df = df.drop(columns=['ID'])
        print("\nDropped column: 'ID'")
    else:
        print("\nColumn 'ID' not found; skipping drop.")
    
    print("\nDataFrame Description (numeric):")
    display(df.describe())
    
    print("\nDataFrame Description (object):")
    display(df.describe(include=['object']))
    
    # optionally show missing values and plot in grid format
    print("\nMissing Values:")
    df_info = pd.DataFrame(df.isnull().sum(), columns=['Missing Values'])
    df_info['Percentage'] = (df_info['Missing Values'] / len(df)) * 100

    # fig, ax = plt.subplots(figsize=(15, 10))
    # df_info['Percentage'].plot(kind='bar', ax=ax, color='tab:blue')
    # ax.set_title('Missing Values in Each Column')
    # ax.set_ylabel('Percentage (%)')
    # ax.set_xlabel('Columns')
    # plt.xticks(rotation=45, ha='right')
    # plt.tight_layout()
    # plt.show()
    
    
    #siwich column into rows for better display

    df_info = df_info.transpose()
    display(df_info)
    # display(df.isnull().sum())
    
    # check the distribution of target column
    print(df['TARGET'].value_counts())
    return df

df = explore_data(df)

DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 355190 entries, 0 to 355189
Columns: 116 entries, ID to TARGET
dtypes: float64(94), int64(9), object(13)
memory usage: 314.3+ MB

Dropped column: 'ID'

DataFrame Description (numeric):


,CR_PROD_CNT_IL,AMOUNT_RUB_CLO_PRC,PRC_ACCEPTS_A_EMAIL_LINK,APP_REGISTR_RGN_CODE,PRC_ACCEPTS_A_POS,PRC_ACCEPTS_A_TK,TURNOVER_DYNAMIC_IL_1M,CNT_TRAN_AUT_TENDENCY1M,SUM_TRAN_AUT_TENDENCY1M,AMOUNT_RUB_SUP_PRC,...,REST_DYNAMIC_CC_3M,MED_DEBT_PRC_YWZ,LDEAL_ACT_DAYS_PCT_TR3,LDEAL_ACT_DAYS_PCT_AAVG,LDEAL_DELINQ_PER_MAXYWZ,TURNOVER_DYNAMIC_CC_3M,LDEAL_ACT_DAYS_PCT_TR,LDEAL_ACT_DAYS_PCT_TR4,LDEAL_ACT_DAYS_PCT_CURR,TARGET
count,355190.000000,316867.000000,155163.0,60550.000000,155163.0,155163.0,355190.000000,77112.000000,77112.000000,316867.000000,...,355190.000000,95713.000000,93448.000000,98175.000000,95713.000000,355190.000000,93448.000000,93448.000000,93448.000000,355190.000000
mean,0.105225,0.044045,0.0,50.947498,0.0,0.0,0.001305,0.416896,0.414572,0.085249,...,0.007309,0.055074,0.025707,0.049943,0.009252,0.004309,0.013938,0.013938,0.013938,0.081435
std,0.431372,0.108449,0.0,21.777855,0.0,0.0,0.029118,0.316493,0.338612,0.142310,...,0.066681,0.215909,0.115732,0.185830,0.092789,0.059852,0.097099,0.097099,0.097099,0.273503
min,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.006944,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.0,33.000000,0.0,0.0,0.000000,0.166667,0.139645,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.0,54.000000,0.0,0.0,0.000000,0.300000,0.285714,0.027117,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.036608,0.0,72.000000,0.0,0.0,0.000000,0.571429,0.661195,0.110005,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,11.000000,1.000000,0.0,89.000000,0.0,0.0,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000



DataFrame Description (object):


,CLNT_TRUST_RELATION,APP_MARITAL_STATUS,APP_KIND_OF_PROP_HABITATION,CLNT_JOB_POSITION_TYPE,CLNT_JOB_POSITION,APP_DRIVING_LICENSE,APP_EDUCATION,APP_TRAVEL_PASS,APP_CAR,APP_POSITION_TYPE,APP_EMP_TYPE,APP_COMP_TYPE,PACK
count,69421,68234,59361,44781,210811,57257,68104,57257,57256,60545,67362,67362,355190
unique,21,13,5,4,19588,2,17,2,2,4,4,4,12
top,FRIEND,M,SO,SPECIALIST,ДИРЕКТОР,N,H,N,N,SPECIALIST,PRIVATE,PRIVATE,102
freq,24896,30724,28056,25123,11200,36332,42459,52750,32843,36622,59087,59087,116986



Missing Values:


,CR_PROD_CNT_IL,AMOUNT_RUB_CLO_PRC,PRC_ACCEPTS_A_EMAIL_LINK,APP_REGISTR_RGN_CODE,PRC_ACCEPTS_A_POS,PRC_ACCEPTS_A_TK,TURNOVER_DYNAMIC_IL_1M,CNT_TRAN_AUT_TENDENCY1M,SUM_TRAN_AUT_TENDENCY1M,AMOUNT_RUB_SUP_PRC,...,REST_DYNAMIC_CC_3M,MED_DEBT_PRC_YWZ,LDEAL_ACT_DAYS_PCT_TR3,LDEAL_ACT_DAYS_PCT_AAVG,LDEAL_DELINQ_PER_MAXYWZ,TURNOVER_DYNAMIC_CC_3M,LDEAL_ACT_DAYS_PCT_TR,LDEAL_ACT_DAYS_PCT_TR4,LDEAL_ACT_DAYS_PCT_CURR,TARGET
Missing Values,0.0,38323.000000,200027.000000,294640.000000,200027.000000,200027.000000,0.0,278078.000000,278078.000000,38323.000000,...,0.0,259477.000000,261742.000000,257015.000000,259477.000000,0.0,261742.000000,261742.000000,261742.000000,0.0
Percentage,0.0,10.789437,56.315493,82.952786,56.315493,56.315493,0.0,78.289929,78.289929,10.789437,...,0.0,73.053014,73.690701,72.359864,73.053014,0.0,73.690701,73.690701,73.690701,0.0


TARGET
0    326265
1     28925
Name: count, dtype: int64


In [4]:
326265/df.shape[0]

0.918564711844365

In [5]:
from scipy.stats import chi2_contingency
import numpy as np

def cramers_v(x, y):
    """Measure association between categorical x and binary y (TARGET)"""
    confusion_matrix = pd.crosstab(x, y)
    chi2 = chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum().sum()
    phi2 = chi2 / n
    r, k = confusion_matrix.shape
    return np.sqrt(phi2 / min(k-1, r-1))

In [6]:
# Annalyze data whitch to keep or drop
print("\nAnalyzing columns with >70% missing values for correlation with TARGET...")
print(f"df shape before dropping columns: {df.shape}")
high_missing = df.columns[df.isnull().sum() / len(df) > TRESHOLD]
for col in high_missing:
    if col == 'TARGET':
        continue
    # coerce to numeric where possible, compute correlation with TARGET
    
    if df[col].dtype == 'object':
        # categorical column
        corr = cramers_v(df[col].dropna(), df.loc[df[col].notnull(), 'TARGET'])
        if corr < 0.1:
            print(f"Dropping column {col} due to low correlation ({corr:.3f}) with TARGET")
            df = df.drop(columns=[col])
    else:
        # numerical column
        series_num = pd.to_numeric(df[col], errors='coerce')
        corr = series_num.corr(df['TARGET'])
        if pd.notnull(corr) and abs(corr) < 0.05:
            print(f"Dropping column {col} due to low correlation ({corr:.3f}) with TARGET")
            df = df.drop(columns=[col])
    
    
    # series_num = pd.to_numeric(df[col], errors='coerce')
    
    
    
    # corr = cramers_v(df[col].dropna(), df.loc[df[col].notnull(), 'TARGET'])
    # missing_pct = df[col].isnull().sum() / len(df) * 100
    # corr_str = f"{corr:.3f}" if pd.notnull(corr) else "N/A"
    # print(f"{col}: Missing={missing_pct:.1f}%, Correlation={corr_str}")
print(f"df shape after dropping columns: {df.shape}")


Analyzing columns with >70% missing values for correlation with TARGET...
df shape before dropping columns: (355190, 115)
Dropping column APP_REGISTR_RGN_CODE due to low correlation (-0.028) with TARGET
Dropping column CLNT_TRUST_RELATION due to low correlation (0.029) with TARGET
Dropping column APP_MARITAL_STATUS due to low correlation (0.030) with TARGET
Dropping column APP_KIND_OF_PROP_HABITATION due to low correlation (0.008) with TARGET
Dropping column CLNT_JOB_POSITION_TYPE due to low correlation (0.036) with TARGET
Dropping column APP_DRIVING_LICENSE due to low correlation (0.031) with TARGET
Dropping column APP_EDUCATION due to low correlation (0.058) with TARGET
Dropping column APP_TRAVEL_PASS due to low correlation (0.024) with TARGET
Dropping column APP_CAR due to low correlation (0.028) with TARGET
Dropping column APP_POSITION_TYPE due to low correlation (0.034) with TARGET
Dropping column APP_EMP_TYPE due to low correlation (0.030) with TARGET
Dropping column DEAL_YQZ_IR

In [7]:
for col in df:
    column_type = df[col].dtype
    if column_type != 'int64' and column_type != 'float64':
        print(f"{col}: {column_type}")

CLNT_JOB_POSITION: object
PACK: object


In [8]:
import numpy as np


def normalize_categorical_columns(df):
    """
    Normalize categorical columns by converting to lowercase/uppercase
    and stripping whitespace
    """
    cat_cols = df.select_dtypes(include=['object']).columns.tolist()
    
    for col in cat_cols:
        # Convert to string, lowercase, and strip whitespace
        df[col] = df[col].astype(str).str.upper()
        df[col] = df[col].str.strip()
        df[col] = df[col].replace('NAN', np.nan)  # Replace 'NA' strings with NaN
        df[col] = df[col].replace('', np.nan)  # Replace empty strings with NaN
    
    return df
df = normalize_categorical_columns(df)

In [9]:
for col in df:
    column_type = df[col].dtype
    if column_type != 'int64' and column_type != 'float64':
        print(f"{col}: {column_type}")


# Count non-null occurrences of each job position
existing_values = {}
for val in df['PACK']:
    if pd.isnull(val):
        continue
    existing_values[val] = existing_values.get(val, 0) + 1
corr = cramers_v(df['PACK'].dropna(), df.loc[df['PACK'].notnull(), 'TARGET'])
missing_pct = df['PACK'].isnull().sum() / len(df) * 100
print(f"PACK: Missing={missing_pct:.1f}%, Correlation={corr:.3f}")
print(f"existing_values length: {len(existing_values)}")
existing_values

CLNT_JOB_POSITION: object
PACK: object
PACK: Missing=0.0%, Correlation=0.071
existing_values length: 12


{'K01': 77083,
 '102': 116986,
 '105': 44936,
 'O01': 50478,
 '103': 24860,
 '101': 1816,
 '107': 27952,
 '301': 4208,
 '104': 6776,
 '108': 2,
 '109': 86,
 'M01': 7}

In [10]:
import category_encoders as ce
import pandas as pd

def encode_categorical_data(df, target):
    """
    Convert object columns to appropriate numeric representations.
    Note: For production, split train/test BEFORE calling this function
    and fit encoders only on training data.
    """
    print("Original shape:", df.shape)
    
    # Get object columns
    obj_cols = df.select_dtypes(include=['object']).columns.tolist()
    print(f"Object columns: {obj_cols}")
    
    # Try to convert object columns to numeric where possible
    for col in obj_cols:
        try:
            converted = pd.to_numeric(df[col], errors='coerce')
            if converted.notna().mean() > 0.9:
                df[col] = converted
                print(f"Converted {col} to numeric")
        except:
            continue
    
    # Update object columns list after numeric conversion
    obj_cols = df.select_dtypes(include=['object']).columns.tolist()
    
    # Separate columns by cardinality
    cat_cols_to_encode = []  # Low cardinality (one-hot)
    high_card_cols = []      # High cardinality (target encode)
    cols_to_drop = []        # All null columns
    
    for col in obj_cols:
        # if col == 'TARGET':
        #     continue
            
        # Skip columns with all nulls
        if df[col].isnull().all():
            print(f"Skipping {col} (all null)")
            cols_to_drop.append(col)
            continue
        
        unique_count = df[col].nunique(dropna=True)
        
        # Categorize by cardinality
        if unique_count < 10:
            cat_cols_to_encode.append(col)
            print(f"Low cardinality {col}: {unique_count} unique values → One-hot encode")
        else:
            high_card_cols.append(col)
            print(f"High cardinality {col}: {unique_count} unique values → Target encode")
    
    # Drop all-null columns
    if cols_to_drop:
        df = df.drop(columns=cols_to_drop)
        print(f"Dropped all-null columns: {cols_to_drop}")
    
    # Target encode high-cardinality columns
    if high_card_cols:
        print(f"\nApplying Target Encoding to: {high_card_cols}")
        
        for col in high_card_cols:
            try:
                # Create mask for valid (non-null) values
                valid_mask = df[col].notna() & target['TARGET'].notna()
                
                # Initialize encoder
                encoder = ce.TargetEncoder(cols=[col], smoothing=1.0)
                
                # Fit and transform on valid data
                df.loc[valid_mask, f"{col}_ENCODED"] = encoder.fit_transform(
                    df.loc[valid_mask, [col]], 
                    target.loc[valid_mask, 'TARGET']
                )[col]
                
                # For null values, use global mean of target
                global_mean = target['TARGET'].mean()
                df[f"{col}_ENCODED"].fillna(global_mean, inplace=True)
                
                print(f"✓ Target encoded: {col}")
                
            except Exception as e:
                print(f"✗ Target encoding failed for {col}: {e}")
                continue
        
        # Drop original high-cardinality columns after encoding
        df = df.drop(columns=high_card_cols)
    
    # One-hot encode low-cardinality columns
    if cat_cols_to_encode:
        print(f"\nOne-hot encoding: {cat_cols_to_encode}")
        df = pd.get_dummies(df, columns=cat_cols_to_encode, drop_first=True)
    
    print(f"\nFinal shape: {df.shape}")
    return df


In [13]:

# Test conversion
ds = encode_categorical_data(df, df)



for col in ds:
    column_type = ds[col].dtype
    if column_type != 'int64' and column_type != 'float64':
        print(f"{col}: {column_type}")


# Count non-null occurrences of each job position
existing_values = {}
for val in ds['CLNT_JOB_POSITION_ENCODED']:
    if pd.isnull(val):
        continue
    existing_values[val] = existing_values.get(val, 0) + 1
corr = cramers_v(ds['CLNT_JOB_POSITION_ENCODED'].dropna(), ds.loc[ds['CLNT_JOB_POSITION_ENCODED'].notnull(), 'TARGET'])
missing_pct = ds['CLNT_JOB_POSITION_ENCODED'].isnull().sum() / len(ds) * 100
print(f"CLNT_JOB_POSITION: Missing={missing_pct:.1f}%, Correlation={corr:.3f}")
print(f"existing_values length: {len(existing_values)}")
existing_values


Original shape: (355190, 88)
Object columns: ['CLNT_JOB_POSITION', 'PACK']
High cardinality CLNT_JOB_POSITION: 15111 unique values → Target encode
High cardinality PACK: 12 unique values → Target encode

Applying Target Encoding to: ['CLNT_JOB_POSITION', 'PACK']
✓ Target encoded: CLNT_JOB_POSITION


/tmp/ipykernel_36374/970197769.py:79: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[f"{col}_ENCODED"].fillna(global_mean, inplace=True)
/tmp/ipykernel_36374/970197769.py:79: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True

✓ Target encoded: PACK

Final shape: (355190, 88)
CLNT_JOB_POSITION: Missing=0.0%, Correlation=0.256
existing_values length: 578


{0.07399577167019028: 1892,
 0.08143528815563501: 144379,
 0.05435549908230976: 7083,
 0.07851108013717945: 9558,
 0.06618883638984543: 24974,
 0.06470588235294118: 3570,
 0.06639839034205232: 497,
 0.07441860465116279: 1075,
 0.07459189252518941: 12803,
 0.07851112055121942: 425,
 0.0949367088607595: 316,
 0.0821917808219178: 365,
 0.03076923076923077: 130,
 0.10877513711151737: 2188,
 0.10571428571428572: 350,
 0.049310710498409335: 1886,
 0.04025774894004517: 75,
 0.05029585798816568: 676,
 0.11743119266055047: 1635,
 0.05740343347639485: 1864,
 0.035728637625968965: 168,
 0.027777777777777776: 72,
 0.07633587786259542: 262,
 0.07851108219176708: 7194,
 0.07851109393697127: 885,
 0.14788732394366197: 142,
 0.07851105937077403: 1350,
 0.0748663101604278: 187,
 0.07851239669421488: 242,
 0.08379888268156424: 179,
 0.06699928724162509: 1403,
 0.08706467661691543: 804,
 0.06784386617100371: 1076,
 0.055842812823164424: 967,
 0.04819277108433735: 1245,
 0.06418383518225039: 1262,
 0.1163

In [14]:
# split data train test


y = pd.DataFrame()
y['TARGET'] = df['TARGET']
X = df.drop(columns=['TARGET'])
X_train, X_test, y_train, y_test = train_test_split(
                            X,y,
                            test_size=0.2,
                            random_state=42,
                            stratify=y)

In [18]:
# Encode categorical data for train and test sets separately
X_train_encoded = encode_categorical_data(X_train.copy(), y_train)
print("*"*80)
X_test_encoded = encode_categorical_data(X_test.copy(), y_test)

# Prepare balanced training by filling nan by mean value of the column
print("Balancing x training set using SMOTE...")

for col in X_train_encoded.columns:
    if X_train_encoded[col].isnull().any():
        mean_value = X_train_encoded[col].mean()
        X_train_encoded[col].fillna(mean_value, inplace=True)

Original shape: (284152, 89)
Object columns: ['CLNT_JOB_POSITION', 'PACK']
High cardinality CLNT_JOB_POSITION: 14051 unique values → Target encode
High cardinality PACK: 12 unique values → Target encode

Applying Target Encoding to: ['CLNT_JOB_POSITION', 'PACK']
✓ Target encoded: CLNT_JOB_POSITION


/tmp/ipykernel_36374/970197769.py:79: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[f"{col}_ENCODED"].fillna(global_mean, inplace=True)
/tmp/ipykernel_36374/970197769.py:79: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True

✓ Target encoded: PACK

Final shape: (284152, 87)
********************************************************************************
Original shape: (71038, 89)
Object columns: ['CLNT_JOB_POSITION', 'PACK']
High cardinality CLNT_JOB_POSITION: 6587 unique values → Target encode
High cardinality PACK: 12 unique values → Target encode

Applying Target Encoding to: ['CLNT_JOB_POSITION', 'PACK']
✓ Target encoded: CLNT_JOB_POSITION
✓ Target encoded: PACK

Final shape: (71038, 87)
Balancing x training set using SMOTE...


/tmp/ipykernel_36374/970197769.py:79: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[f"{col}_ENCODED"].fillna(global_mean, inplace=True)
/tmp/ipykernel_36374/2369109394.py:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=Tru

In [19]:
# Balence Dataset for training set only

print("Balancing training set using SMOTE...")
print(f"Before SMOTE: {y_train['TARGET'].value_counts()} | X shape: {X_train_encoded.shape}")
smote = SMOTE(sampling_strategy='minority', random_state=42)
X_train_balanced, y_train_balanced = smote.fit_resample(X_train_encoded, y_train)
print(f"After SMOTE: {y_train_balanced['TARGET'].value_counts()} | X shape: {X_train_balanced.shape}")

Balancing training set using SMOTE...
Before SMOTE: TARGET
0    261012
1     23140
Name: count, dtype: int64 | X shape: (284152, 87)
After SMOTE: TARGET
0    261012
1    261012
Name: count, dtype: int64 | X shape: (522024, 87)
